# IG to Test Kit FULL pipeline

## Setup

### Importing Notebooks as Modules (from the [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/4.x/examples/Notebook/rstversions/Importing%20Notebooks.html))

In [2]:
import inspect
import json
import llm_utils
import importlib

## Initializing LLM Clients

In [3]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [4]:
llm_clients.clients

{'claude': <anthropic.Anthropic at 0x1153f1ac0>,
 'gemini': genai.GenerativeModel(
     model_name='models/gemini-2.5-pro',
     generation_config={'max_output_tokens': 8192, 'temperature': 0.3},
     safety_settings={<HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
     tools=None,
     system_instruction=None,
     cached_content=None
 ),
 'gpt': <openai.OpenAI at 0x11501a9f0>}

## Text Extraction

### HTML to Markdown Conversion Using Markdownify (Langchain Tool)

In [4]:
import HTML_extractor

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
urls = [
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/index.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/ChangeHistory.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/examples.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/implementation.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/profiles.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/artifacts.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/CapabilityStatement-plan-net.html"
]

HTML_extractor.convert_urls_to_markdown(urls, output_dir="text_extraction/PlanNet/site/markdown")

Fetching pages: 100%|##########| 7/7 [00:00<00:00,  7.93it/s]


Created: index.md
Created: ChangeHistory.md
Created: examples.md
Created: implementation.md
Created: profiles.md
Created: artifacts.md
Created: CapabilityStatement_plan_net.md


### Markdown Post-processing

In [7]:
import markdown_cleaner
markdown_cleaner.process_directory("checkpoints/text_extraction/PlanNet/site/markdown", "checkpoints/post_processing/")

Found 7 markdown files in checkpoints/text_extraction/PlanNet/site/markdown
Cleaned and saved: checkpoints/post_processing/implementation.md
Cleaned and saved: checkpoints/post_processing/examples.md
Cleaned and saved: checkpoints/post_processing/profiles.md
Cleaned and saved: checkpoints/post_processing/ChangeHistory.md
Cleaned and saved: checkpoints/post_processing/artifacts.md
Cleaned and saved: checkpoints/post_processing/index.md
Cleaned and saved: checkpoints/post_processing/CapabilityStatement_plan_net.md

Processing complete: 7 files successfully cleaned, 0 failed


## Requirements Extraction

### Prompt-based Requirement Extraction

In [4]:
importlib.reload(reqs_extraction)

NameError: name 'reqs_extraction' is not defined

In [5]:
import reqs_extraction

INFO:root:Current working directory: /Users/ceadams/Documents/onclaive/onclaive/pipeline
INFO:root:Project root: /Users/ceadams/Documents/onclaive/onclaive
INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:root:Using prompts directory: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:root:Requirements extraction prompt: /Users/ceadams/Documents/onclaive/onclaive/prompts/requirements_extraction.md


In [6]:
reqs_extraction.run_requirements_extractor(
    '/Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown_safety', 
    'checkpoints/requirements_extraction/markdown', 
    'gemini', 
    llm_clients)

INFO:root:Found markdown directory at /Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown_safety
INFO:root:Found 2 markdown files
INFO:root:Processing with gemini...
INFO:root:Starting processing with gemini on directory: /Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown_safety
INFO:root:Found 2 markdown files:
INFO:root:  - CapabilityStatement_plan_net.md
INFO:root:  - StructureDefinition-qualification-definitions.md
INFO:root:Organized 2 files into 2 processing groups
INFO:root:Processing single file: CapabilityStatement_plan_net.md
INFO:root:Split CapabilityStatement_plan_net.md into 1 chunks using dynamic sizing
INFO:root:Processing chunk 1/1 of CapabilityStatement_plan_net.md



Processing Implementation Guide with Gemini...
This may take several minutes depending on the size of the Implementation Guide.

⚠️  SAFETY FILTER BLOCKED CONTENT #1
BLOCKED CONTENT SAMPLE:
System: You are a Healthcare Integration Test Engineer with expertise in INCOSE Systems Engineering standards, analyzing FHIR 
    Implementation Guide content to identify and format testable requirements following INCOSE specifications.

User: [{'parts': [{'text': '# ABOUT THIS TASK\nYou are analyzing chunk 1 of 1 from a FHIR Implementation Guide. Your task is to:\n1. Extract specific, testable requirements\n2. Format them according to INCOSE Systems Engineering standards\n3. Create requirement...
Skipping this chunk and continuing...



INFO:root:Processing single file: StructureDefinition-qualification-definitions.md
INFO:root:Split StructureDefinition-qualification-definitions.md into 1 chunks using dynamic sizing
INFO:root:Processing chunk 1/1 of StructureDefinition-qualification-definitions.md



⚠️  SAFETY FILTER BLOCKED CONTENT #2
BLOCKED CONTENT SAMPLE:
System: You are a Healthcare Integration Test Engineer with expertise in INCOSE Systems Engineering standards, analyzing FHIR 
    Implementation Guide content to identify and format testable requirements following INCOSE specifications.

User: [{'parts': [{'text': '# ABOUT THIS TASK\nYou are analyzing chunk 1 of 1 from a FHIR Implementation Guide. Your task is to:\n1. Extract specific, testable requirements\n2. Format them according to INCOSE Systems Engineering standards\n3. Create requirement...
Skipping this chunk and continuing...



INFO:root:Completed processing 2 files
INFO:root:Generated requirements document saved to checkpoints/requirements_extraction/markdown/gemini_reqs_list_v1_20250723_150027.md



Processing complete!
Generated requirements document: checkpoints/requirements_extraction/markdown/gemini_reqs_list_v1_20250723_150027.md
Processed 2 files


### RAG-based Requirement Extraction

This extraction requirement extraction method differs from the first in that, as a part of the creation of its prompt, it performs a semantic search on example sections of FHIR IG text and the human-generated requirements that were produced in reference to those sections of text to find the most similar section(s) of FHIR IG text in the database and their associated requirement(s). Those sets of IG text and requirement(s) are then supplied to the LLM as few-shot examples

In [3]:
import rag_reqs_extraction

In [ ]:
importlib.reload(rag_reqs_extraction)
rag_reqs_extraction.full_pass(llm_clients, 'claude', "checkpoints/post_processing/", "checkpoints/requirements_extraction/RAG")

## Requirement Downselection

In [24]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=["checkpoints/requirements_extraction/claude_reqs_list_v1_20250429_081756.md"],
    rag_files=["checkpoints/requirements_extraction/RAG/plan_net_reqs.json"]
    )

## Test Plan Generation

In [8]:
import logging
llm_clients.logger.setLevel(logging.INFO)

In [19]:
import req_to_testplan
importlib.reload(req_to_testplan)

result = req_to_testplan.generate_consolidated_test_plan(
    llm_clients,
    'gemini',  # or 'gemini' or 'gpt'
    llm_clients.logger,
    "/Users/ceadams/Documents/onclaive/onclaive/reqs_extraction/revised_reqs_output/small-plan-net-requirements-v3.md",
    "/Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md",
    "Plan-Net IG",
    output_dir='checkpoints/testplan_generation'
)

print(f"Generated test plan with improved capability parsing: {result['test_plan_path']}")

2025-07-01 14:02:01,909 - root - INFO - Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
2025-07-01 14:02:01,910 - llm_utils - INFO - Starting test plan generation with gemini for Plan-Net IG
2025-07-01 14:02:01,911 - llm_utils - INFO - Parsed 5 requirements from /Users/ceadams/Documents/onclaive/onclaive/reqs_extraction/revised_reqs_output/small-plan-net-requirements-v3.md
2025-07-01 14:02:01,912 - llm_utils - INFO - Parsed capability statement from /Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
2025-07-01 14:02:01,913 - llm_utils - INFO - Identifying group for requirement REQ-34 using gemini...
2025-07-01 14:02:12,576 - llm_utils - INFO - Identifying group for requirement REQ-35 using gemini...
2025-07-01 14:02:22,932 - llm_utils - INFO - Identifying group for requirement REQ-36 using gemini...
2025-07-01 14:02:37,736 - llm_utils - INFO - Identifying group for requirement REQ-37 using gemini...
2025

Generated test plan with improved capability parsing: checkpoints/testplan_generation/gemini_test_plan_20250701_140201.md


In [ ]:
capability_statement = req_to_testplan.parse_capability_statement_by_chunks(
    "/Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md"
)

print(f"\nCapability Statement Analysis:")
print(f"- Found {len(capability_statement['resource_list'])} resources: {capability_statement['resource_list']}")
print(f"- General capabilities text length: {len(capability_statement['general_capabilities'])} characters")



Capability Statement Analysis:
- Found 8 resources: ['Endpoint', 'HealthcareService', 'InsurancePlan', 'Location', 'Organization', 'OrganizationAffiliation', 'Practitioner', 'PractitionerRole']
- General capabilities text length: 1113 characters


In [ ]:
sample_requirement = {
    'summary': 'Organization search by name',
    'description': 'The Plan-Net design is based around the following types of searches: Organization by Name - Example: Montgomery Cardiology or CVS - Focal Resource and Field: Organization.name - Qualifications of Search: Location, network, specialty.',
    'verification': 'Automatically testable',
    'actor': 'Health Plan API'
}

relevant_info = req_to_testplan.extract_relevant_capability_chunks(sample_requirement, capability_statement)
print(f"\nSample requirement matching:")
print(f"- Relevant capability info length: {len(relevant_info)} characters")
print(f"- Preview: {relevant_info}...")


Sample requirement matching:
- Relevant capability info length: 6082 characters
- Preview: ### Applicable Capability Statement Information

#### Relevant Resource Capabilities

#### Location

Conformance Expectation: **SHALL**

Supported Profiles:

 [Plan\-Net Location](StructureDefinition-plannet-Location.html)

Reference Policy: `resolves`

Profile Interaction Summary:

* **SHALL** support 
 `search-type`, 
 `read`.
* **SHOULD** support 
 `vread`.

Fetch and Search Criteria:

* A Server **SHALL** be capable of returning a Location resource using:

`GET [base]/Location/[id]`

* A Server **SHOULD** be capable of returning a Location resource using:

`GET [base]/Location/[id]/_history/vid`

 * A Server **SHALL** be capable of supporting the following \_includes:

 Location:endpoint \- `GET [base]/Location?[parameter=value]&_include=Location:endpoint`  

 Location:organization \- `GET [base]/Location?[parameter=value]&_include=Location:organization`  

 Location:partof \- `GET [base]/Lo

### RAG Approach

In [22]:
import warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set logging level to reduce noise
import logging
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
logging.getLogger("backoff").setLevel(logging.ERROR)

import req_to_testplan_rag
importlib.reload(req_to_testplan_rag)

req_to_testplan_rag.clear_capability_collection("capability_statements")


2025-07-02 13:55:41,483 - root - INFO - Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts


Deleted existing collection: capability_statements


2025-07-02 13:55:46,426 - backoff - ERROR - Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


In [26]:

req_to_testplan_rag.generate_consolidated_test_plan(
    llm_clients, 
    'gemini', 
    llm_clients.logger, 
    "/Users/ceadams/Documents/onclaive/onclaive/reqs_extraction/revised_reqs_output/small-plan-net-requirements-v4.md", 
    "../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md", 
    "Plan-Net IG",
    output_dir='checkpoints/testplan_generation',
    verbose=True)

2025-07-09 12:24:19,465 - llm_utils - INFO - Starting test plan generation with gemini for Plan-Net IG
2025-07-09 12:24:19,467 - llm_utils - INFO - Parsed 5 requirements from /Users/ceadams/Documents/onclaive/onclaive/reqs_extraction/revised_reqs_output/small-plan-net-requirements-v4.md
2025-07-09 12:24:19,490 - req_to_testplan_rag - INFO - Using existing collection: capability_statements
2025-07-09 12:24:19,494 - llm_utils - INFO - Initialized capability collection from ../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
2025-07-09 12:24:19,504 - llm_utils - INFO - Identifying group for requirement REQ-02 using gemini...
2025-07-09 12:24:25,258 - backoff - ERROR - Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (


Processing Group: General Requirements (1 requirements)

Processing REQ-09: Client processing of missing information indicators

RAG RETRIEVAL FOR REQ-09
Query: Client processing of missing information indicators "Consumer App actors SHALL be able to process resource instances containing Must Support data elements asserting missing information." Application Actor FHIR SHALL
Searching for 5 most relevant capability chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Found 5 matching chunks:

  Match 1 (distance: 1.0090609788894653):
  Length: 802 chars
  Preview: FHIR Major Section: ## Plan\-Net CapabilityStatement

* Implementation Guide Version: 1\.0\.0
* FHIR...
  ...local use cases and other contextual requirements.

  Match 2 (distance: 1.1007232666015625):
  Length: 3225 chars
  Preview: FHIR Resource Detail: #### Organization

Conformance Expectation: **SHALL**

Supported Profiles:

 [...
  ...Organization?coverage-area=[coverage-area]` |

---

  Match 3 (distance: 1.1312487125396729):
  Length: 996 chars
  Preview: FHIR Major Section: ## CapabilityStatement: Plan\-Net CapabilityStatement

| *Official URL*: http://...
  ...openapi.json) \| [Download](plan-net.openapi.json)

  Match 4 (distance: 1.1321117877960205):
  Length: 2302 chars
  Preview: FHIR Resource Detail: #### InsurancePlan

Conformance Expectation: **SHALL**

Supported Profiles:

 ...
  ... [base]/InsurancePlan?type=[system]|[code]` |

---

  Match 5 (distance: 1.140368461608

2025-07-09 12:27:23,446 - llm_utils - INFO - Processing requirement for group 'Network': REQ-06
2025-07-09 12:27:23,446 - llm_utils - INFO - Generating test specification for REQ-06 using gemini...


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Found 5 matching chunks:

  Match 1 (distance: 0.7856061458587646):
  Length: 3288 chars
  Preview: FHIR Resource/Component: ### RESTful Capabilities by Resource/Profile:

**Summary of Search Criteria...
  ...nerRole:network, PractitionerRole:endpoint |  |  |

  Match 2 (distance: 0.7965418696403503):
  Length: 3036 chars
  Preview: FHIR Resource Detail: #### HealthcareService

Conformance Expectation: **SHALL**

Supported Profiles...
  ...lthcareService?_lastUpdated=[_lastUpdated]` |

---

  Match 3 (distance: 0.8781458139419556):
  Length: 1169 chars
  Preview: FHIR Resource/Component: ### FHIR RESTful Capabilities

The Plan\-Net Server **SHALL**:

1. Support ...
  ...eturning an `HTTP 401` unauthorized response code.

  Match 4 (distance: 0.8851763010025024):
  Length: 3225 chars
  Preview: FHIR Resource Detail: #### Organization

Conformance Expectation: **SHALL**

Supported Profiles:

 [...
  ...Organization?coverage-area=[coverage-area]` |

---

  Match 5 (distance: 0.9248039722

2025-07-09 12:30:10,650 - llm_utils - INFO - Consolidated test plan saved to checkpoints/testplan_generation/gemini_test_plan_20250709_122419.md


Completed test specification for REQ-10


{'requirements_count': 5,
 'group_count': 3,
 'test_plan_path': 'checkpoints/testplan_generation/gemini_test_plan_20250709_122419.md'}

In [24]:
import chromadb
from chromadb.utils import embedding_functions
from datetime import datetime
from pathlib import Path


def inspect_capability_chunks(output_file=None):
    """
    Inspect all chunks stored in the ChromaDB collection and save to markdown file
    """
    # Connect to ChromaDB
    chroma_client = chromadb.Client()
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-mpnet-base-v2"
    )
    
    try:
        # Get the existing collection
        collection = chroma_client.get_collection('capability_statements', embedding_function=sentence_transformer_ef)
        
        # Get all documents
        all_docs = collection.get()
        
        # Create output file name if not provided
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"capability_chunks_analysis_{timestamp}.md"
        
        # Prepare markdown content
        markdown_content = []
        
        # Header
        markdown_content.append("# Capability Statement Chunking Analysis")
        markdown_content.append(f"*Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n")
        
        # Summary statistics
        chunk_lengths = [len(doc) for doc in all_docs['documents']]
        markdown_content.append("## 📊 Summary Statistics")
        markdown_content.append(f"- **Total chunks**: {len(all_docs['documents'])}")
        markdown_content.append(f"- **Average chunk length**: {sum(chunk_lengths) / len(chunk_lengths):.1f} characters")
        markdown_content.append(f"- **Shortest chunk**: {min(chunk_lengths)} characters")
        markdown_content.append(f"- **Longest chunk**: {max(chunk_lengths)} characters\n")
        
        # Chunk type analysis
        chunk_types = {
            'Resource sections': 0,
            'Table sections': 0,
            'General sections': 0,
            'Other': 0
        }
        
        for doc in all_docs['documents']:
            if 'FHIR' in doc and 'Resource:' in doc:
                chunk_types['Resource sections'] += 1
            elif 'FHIR Capability Table:' in doc:
                chunk_types['Table sections'] += 1
            elif 'FHIR Capability Statement:' in doc:
                chunk_types['General sections'] += 1
            else:
                chunk_types['Other'] += 1
        
        markdown_content.append("## 🔍 Chunk Type Analysis")
        for chunk_type, count in chunk_types.items():
            markdown_content.append(f"- **{chunk_type}**: {count} chunks")
        markdown_content.append("")
        
        # Table of contents
        markdown_content.append("## 📋 Table of Contents")
        for i in range(len(all_docs['documents'])):
            doc = all_docs['documents'][i]
            # Create a short preview for TOC
            preview = doc[:50].replace('\n', ' ').strip()
            if len(doc) > 50:
                preview += "..."
            markdown_content.append(f"- [Chunk {i+1}](#chunk-{i+1}) ({len(doc)} chars): {preview}")
        markdown_content.append("")
        
        # Detailed chunks
        markdown_content.append("## 📄 Detailed Chunk Analysis")
        
        for i, doc in enumerate(all_docs['documents']):
            # Chunk header
            markdown_content.append(f"### Chunk {i+1}")
            markdown_content.append(f"**Length**: {len(doc)} characters  ")
            markdown_content.append(f"**ID**: {all_docs['ids'][i]}  ")
            
            # Determine chunk type
            chunk_type = "Other"
            if 'FHIR' in doc and 'Resource:' in doc:
                chunk_type = "Resource Section"
            elif 'FHIR Capability Table:' in doc:
                chunk_type = "Table Section"
            elif 'FHIR Capability Statement:' in doc:
                chunk_type = "General Section"
            
            markdown_content.append(f"**Type**: {chunk_type}\n")
            
            # Content in code block for better formatting
            markdown_content.append("**Content:**")
            markdown_content.append("```")
            markdown_content.append(doc)
            markdown_content.append("```\n")
            
            markdown_content.append("---\n")
        
        # Write to file
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(markdown_content))
        
        print(f"✅ Chunk analysis saved to: {output_file}")
        print(f"📊 Total chunks analyzed: {len(all_docs['documents'])}")
        
        # Also print summary to console
        print(f"\n📋 QUICK SUMMARY:")
        print(f"   Total chunks: {len(all_docs['documents'])}")
        for chunk_type, count in chunk_types.items():
            print(f"   {chunk_type}: {count}")
            
        return all_docs, output_file
        
    except Exception as e:
        print(f"❌ Error inspecting collection: {e}")
        print("Make sure you've run the script first to create the collection!")
        return None, None

def quick_chunk_preview(output_file=None, num_chunks=5):
    """Create a shorter markdown file with just preview of first few chunks"""
    chroma_client = chromadb.Client()
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-mpnet-base-v2"
    )
    
    try:
        collection = chroma_client.get_collection('capability_statements', embedding_function=sentence_transformer_ef)
        all_docs = collection.get()
        
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"capability_chunks_preview_{timestamp}.md"
        
        markdown_content = []
        markdown_content.append("# Capability Statement Chunks Preview")
        markdown_content.append(f"*Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n")
        markdown_content.append(f"Showing first {num_chunks} chunks out of {len(all_docs['documents'])} total\n")
        
        for i in range(min(num_chunks, len(all_docs['documents']))):
            doc = all_docs['documents'][i]
            markdown_content.append(f"## Chunk {i+1}")
            markdown_content.append(f"**Length**: {len(doc)} characters\n")
            markdown_content.append("```")
            markdown_content.append(doc)
            markdown_content.append("```\n")
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(markdown_content))
        
        print(f"✅ Chunk preview saved to: {output_file}")
        return output_file
        
    except Exception as e:
        print(f"Error: {e}")
        return None

# Run the full analysis and save to markdown
chunks_data, analysis_file = inspect_capability_chunks()

✅ Chunk analysis saved to: capability_chunks_analysis_20250702_135655.md
📊 Total chunks analyzed: 13

📋 QUICK SUMMARY:
   Total chunks: 13
   Resource sections: 0
   Table sections: 0
   General sections: 0
   Other: 13


2025-07-02 13:57:00,559 - backoff - ERROR - Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


## Test Kit Generation

In [8]:
import plan_to_tests
importlib.reload(plan_to_tests)

plan_to_tests.generate_inferno_test_kit(
    llm_clients,
    'claude',
    '/Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/test_plan_output/example_claude_test_plan_20250416_143409.md',
    #'../test_kit_dev/inferno-guidance.md',
    output_dir='checkpoints/testkit_generation',
    expected_actors=["Health Plan API Actor", "Application Actor"]
)

2025-06-19 15:10:47,505 - plan_to_tests - INFO - Starting Inferno test generation with claude for PlanNet
2025-06-19 15:10:47,516 - plan_to_tests - INFO - Parsed test plan into 11 sections
2025-06-19 15:10:47,516 - plan_to_tests - INFO - Found 11 total requirements
2025-06-19 15:10:47,518 - plan_to_tests - INFO - Loaded Inferno DSL guidance
2025-06-19 15:10:47,518 - plan_to_tests - INFO - Processing section: Application-Level Requirements with 1 requirements
2025-06-19 15:10:47,518 - plan_to_tests - INFO - Generating tests for section: Application-Level Requirements
2025-06-19 15:10:47,519 - plan_to_tests - INFO - Generating test for requirement: REQ-08
2025-06-19 15:10:47,520 - plan_to_tests - INFO - Requirement REQ-08: Sending 989 tokens to claude API (limit: 16000)


Found 11 potential requirements
Processing requirement: REQ-08
Added requirement REQ-08 to section Application-Level Requirements
Processing requirement: REQ-01
Added requirement REQ-01 to section Authentication
Processing requirement: REQ-09
Added requirement REQ-09 to section Base Requirements
Processing requirement: REQ-07
Added requirement REQ-07 to section CORE Conformance
Processing requirement: REQ-06
Added requirement REQ-06 to section Cross-Resource
Processing requirement: REQ-04
Added requirement REQ-04 to section General Requirements
Processing requirement: REQ-05
Added requirement REQ-05 to section Global
Processing requirement: REQ-11
Added requirement REQ-11 to section OrganizationAffiliation
Processing requirement: REQ-02
Added requirement REQ-02 to section Plan-Net API Security
Processing requirement: REQ-10
Added requirement REQ-10 to section PractitionerRole
Processing requirement: REQ-03
Added requirement REQ-03 to section Security


2025-06-19 15:10:58,370 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-19 15:10:58,371 - plan_to_tests - INFO - Successfully generated test for requirement: REQ-08
2025-06-19 15:10:58,372 - plan_to_tests - INFO - Validating test for requirement: REQ-08
2025-06-19 15:10:58,372 - plan_to_tests - INFO - Validation for test: Sending 1528 tokens to claude API
2025-06-19 15:11:07,548 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-19 15:11:07,552 - plan_to_tests - INFO - Successfully validated test for requirement: REQ-08
2025-06-19 15:11:10,556 - plan_to_tests - INFO - Processing section: Authentication with 1 requirements
2025-06-19 15:11:10,558 - plan_to_tests - INFO - Generating tests for section: Authentication
2025-06-19 15:11:10,562 - plan_to_tests - INFO - Generating test for requirement: REQ-01
2025-06-19 15:11:10,565 - plan_to_tests - INFO - Requirement REQ-01: Sending 983 token

{'total_sections': 11,
 'total_requirements': 11,
 'generated_tests': 11,
 'module_dir': 'checkpoints/testkit_generation/plannet',
 'module_file': 'checkpoints/testkit_generation/plannet.rb'}